In [40]:
import heapq

In [ ]:
with open("input-example.txt") as f:
    lines = f.readlines()
lines = [l.strip() for l in lines]
lines[:5]

In [42]:
start_node = (0, 0)
target_node = (0, 0)
for i in range(0, len(lines)):
    for j in range(0, len(lines[0])):
        if lines[i][j] == "E":
            target_node = (i, j)
        if lines[i][j] == "S":
            start_node = (i, j)

In [43]:
dirs = [(-1, 0), (0, 1), (1, 0), (0, -1)]

# Get dirs to turn to (180 makes no sense)
turn_dict = {
    (-1, 0): [(0, -1), (0, 1)],
    (0, 1): [(-1, 0), (1, 0)],
    (1, 0): [(0, 1), (0, -1)],
    (0, -1): [(1, 0), (-1, 0)],
}


def walk(pos, dir):
    return (pos[0] + dir[0], pos[1] + dir[1])


def in_bounds(pos, field):
    return (
        pos[0] >= 0 and pos[0] < len(field) and pos[1] >= 0 and pos[1] < len(field[0])
    )


def print_field(
    field,
):
    i = 0
    for l in field:
        j = 0
        for p in l:
            print(p, end="")
            j += 1
        print()
        i += 1

In [44]:
facing = (0, 1)

# Nodes are always position and facing direction 
start = (start_node, facing)

visited_nodes = set()

In [45]:
# Dijkstra with cost increase of 1000 when adding a neighbor node after a turn
# We ignore 180 Degree turns as they are always longer and make no sense to add to the graph
def dijkstra(start, target, field):
    priority_queue = []
    heapq.heappush(priority_queue, (0, start))
    prev_nodes = {}
    distances = {}

    distances[start[0]] = 0
    prev_nodes[start[0]] = None

    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)

        if current_node[0] == target:
            break

        if current_distance > distances.get(current_node, float('inf')):
            continue

        # Get neighbours
        neighbours = []
        # print(current_node)
        next_node = walk(current_node[0], current_node[1])
        if field[next_node[0]][next_node[1]] != '#':
            neighbours.append((1, (next_node, current_node[1])))
        
        turn_dirs = turn_dict[current_node[1]]
        for turn_dir in turn_dirs:
            next_node = walk(current_node[0], turn_dir)
            if field[next_node[0]][next_node[1]] != '#':
                neighbours.append((1001, (next_node, turn_dir)))

        for weight, neighbor in neighbours:
            distance = current_distance + weight
            if distance < distances.get(neighbor[0], float('inf')):
                distances[neighbor[0]] = distance
                prev_nodes[start[0]] = current_node[0]
                heapq.heappush(priority_queue, (distance, neighbor))

    return distances, prev_nodes


distances, prev_nodes = dijkstra(start, target_node, lines)
# print(distances, prev_nodes)

In [ ]:
distances[target_node]

In [47]:
# part 2

In [ ]:
# Modify to keep multiple short paths
# Instead of only keeping coordinate as key, keep coordinate and direction facing when entering the tile 
# as those are technically different nodes in a graph as a required turn increases cost
# Then keep a set of all predecessors that have the same dist
def dijkstra_mod(start, target, field):
    priority_queue = []
    heapq.heappush(priority_queue, (0, start))
    prev_nodes = {}
    distances = {}

    distances[start] = 0
    prev_nodes[start] = None

    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)
        
        if current_node[0] == target:
            break

        if current_distance > distances.get(current_node, float('inf')):
            continue

        # Get neighbours 
        neighbours = []
        # When walking straight
        next_node = walk(current_node[0], current_node[1])
        if field[next_node[0]][next_node[1]] != '#':
            neighbours.append((1, (next_node, current_node[1])))
        # Or turning with cost + 1000 and direction after turn as next node
        turn_dirs = turn_dict[current_node[1]]
        for turn_dir in turn_dirs:
            next_node = walk(current_node[0], turn_dir)
            if field[next_node[0]][next_node[1]] != '#':
                neighbours.append((1001, (next_node, turn_dir)))

        for weight, neighbor in neighbours:
            distance = current_distance + weight
            if distance <= distances.get(neighbor, float('inf')):
                distances[neighbor] = distance
                if neighbor not in prev_nodes:
                    prev_nodes[neighbor] = set()
                # If a shorter node is found, reset
                if distance < distances.get(neighbor, float('inf')):
                    prev_nodes[neighbor] = set()
                # Add predecessor, if not reset this means equal len pred found
                prev_nodes[neighbor].add(current_node)
                heapq.heappush(priority_queue, (distance, neighbor))

    return distances, prev_nodes


distances, prev_nodes = dijkstra_mod(start, target_node, lines)
[(k,v) for k,v in distances.items() if k[0] == target_node]

In [ ]:
# From the example, this should have 2 predecessors
# Both are from (7,15) but one needs to turn, because it has direction (-1,0) (trying to walk to the right) depending from where (7,15) was entered
# If only fields without direction are considered, they would differ in cost by 1k because of the not considered turn
# But with direction, they have the same cost connecting them to (6,15)
[(k,v) for k,v in prev_nodes.items() if k[0] == (6,15)]

In [ ]:
# (6,15) has only one distance
[(k,v) for k,v in distances.items() if k[0] == (6,15)]

In [ ]:
# but (7,15) has two distances differing by 1000 cost but with different directions
# When processing the next node (6,15), one gets +1 the other +1001, so they are both valid predecessors
# if checking fields before that, so (8,15) and (7,14), they have the same distances again (and another entry from turning away from the target during the algorithm)
[(k,v) for k,v in distances.items() if k[0] == (7,15)]

In [52]:
# Expand all previous nodes until everything is expanded and return all distinct found nodes
def get_path(target, prev_nodes):
    expanding_nodes = [(k) for k,v in distances.items() if k[0] == target]
    prevs = []
    while expanding_nodes:
        # print(expanding_nodes)
        n = expanding_nodes.pop()

        prev = prev_nodes[n]
        if not prev:
            continue
        prevs.append(n)
        expanding_nodes += prev_nodes[n]
    return {n[0] for n in prevs}
# get_path(target_node, prev_nodes)

In [ ]:
# Places to sit at
len(get_path(target_node, prev_nodes)) + 1 # Start node needs to be considered

In [ ]:
def print_maze(
    field, path_nodes
):
    i = 0
    for l in field:
        j = 0
        for p in l:
            if (i,j) in path_nodes and p not in ['S', 'E']:
                print("O", end="")
            else: 
                print(p, end="")
            j += 1
        print()
        i += 1
print_maze(lines, get_path(target_node, prev_nodes))